# Washington DC Biking data | Hourly Bike Count Prediction
MBD O-1-5

### Notebook preperation

In [1]:
# Imports
%matplotlib inline


from My_Functions import * 
import numpy as np
import pandas as pd
import seaborn as sns
import warnings

from collections import defaultdict

from sklearn import preprocessing
from sklearn.metrics import mean_squared_error, r2_score,roc_curve
from sklearn.model_selection import train_test_split, KFold,StratifiedKFold
from sklearn.model_selection import cross_val_score, cross_val_predict,validation_curve
from sklearn.ensemble import RandomForestRegressor


from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import LabelBinarizer, RobustScaler,PolynomialFeatures
from sklearn.neighbors import KNeighborsRegressor
from scipy import stats
from scipy.stats import skew, boxcox_normmax
from scipy.special import boxcox1p

import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator , MultipleLocator
##from matplotlib import XAxis
from gplearn.genetic import SymbolicRegressor

from sklearn import metrics
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, LabelBinarizer,MinMaxScaler
from sklearn.pipeline import make_pipeline
from sklearn.feature_selection import RFE, RFECV
from sklearn.linear_model import LogisticRegression,LinearRegression, OrthogonalMatchingPursuit
from sklearn.model_selection import train_test_split , TimeSeriesSplit, GridSearchCV
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import roc_auc_score, roc_curve, accuracy_score
from matplotlib.gridspec import GridSpec
import plotly.tools as tls
import plotly
plotly.tools.set_credentials_file(username='Furqan92', api_key='22DfVN5rFRg79OYygN5h')
import plotly.plotly as py
from sklearn.decomposition import PCA
from pandas import DataFrame 
from sklearn.exceptions import ConvergenceWarning
import warnings

warnings.simplefilter(action='ignore')
random_seed = 1234

In [2]:
# To automatically reload the function file 
%load_ext autoreload
%aimport My_Functions
%run My_Functions.py
%autoreload 1

In [3]:
# Data Import
hourly_raw_data = pd.read_csv('hour.csv')

# Feature Engineering

### Converting `dteday` to date

In [4]:
hourly_raw_data['dteday']=pd.to_datetime(hourly_raw_data['dteday'], format='%Y-%m-%d')

## Add `isDaylight` and  `isNoon` for hourly data

In [5]:
hourly_raw_data['isDaylight']=0
hourly_raw_data['isNoon']=0

hourly_raw_data = hourly_raw_data.apply(lambda x: isDaylight(x), axis=1)

### Adding the temp atemp windspeed hum relative to the last 7 days value

In [6]:
to_relative  = ['temp', 'atemp', 'hum','windspeed']
hourly_raw_data = relative_values(hourly_raw_data, to_relative)

### Adding `RushHour-High` & 	`RushHour-Med`  &	`RushHour-Low`

In [7]:
hourly_raw_data['RushHour-High'] = 0
hourly_raw_data['RushHour-Med'] = 0
hourly_raw_data['RushHour-Low'] = 0

hourly_raw_data = hourly_raw_data.apply(lambda x: addRushHourFlags(x), axis=1)

## Splitting Data

In [8]:
wd_hr = num_name(hourly_raw_data.loc[(~hourly_raw_data['weekday'].isin([0,6]) )])
we_hr = num_name(hourly_raw_data.loc[(hourly_raw_data['weekday'].isin([0,6]) )])

### One hot Encoding | 2x for splitted datasets
For `season`, `weathersit`, `mnth`,`weekday`,`hr`

In [9]:
category  = ['season', 'weathersit', 'mnth','weekday','hr']

wd_h = onehot_encode(wd_hr,category)
wd_h  = wd_h.drop('instant',axis=1)

we_h = onehot_encode(we_hr,category)
we_h  = we_h.drop('instant',axis=1)

## Genetic Programming | 2x for splitted datasets

In [10]:
dates = wd_h['dteday']
registered = wd_h['registered']
casual = wd_h['casual']
wd_h = Genetic_P(wd_h.drop(['registered','casual','dteday'],axis=1),'cnt')
wd_h['dteday'] = dates
wd_h['registered'] = registered
wd_h['casual'] = casual

    |    Population Average   |             Best Individual              |
---- ------------------------- ------------------------------------------ ----------
 Gen   Length          Fitness   Length          Fitness      OOB Fitness  Time Left
   0     8.18 0.10375241123439781        3 0.7249914394553755 0.7245790741416474      1.52m
   1     6.33 0.3602701187661421        3 0.727406836383819 0.7019720591681775      1.38m
   2     3.65 0.5958296307178244        9 0.7541645569695625 0.7213500447094823      1.25m
   3     4.05 0.6576251880059677       10 0.7518153092265795 0.7472683310009098      1.13m
   4     5.17 0.6660136599249062       13 0.7644241625749374 0.752409423504761      1.04m
   5     8.18 0.6525590123095099       15 0.7735146491505701 0.7509950897310395     58.17s
   6    10.21 0.6813037434652417       18 0.7808023745013581 0.7713040145135132     54.27s
   7    12.96 0.7116697198032172       18 0.781054839376224 0.7693768744767354     50.54s
   8    16.37 0.7246709504479

In [11]:
dates = we_h['dteday']
registered = we_h['registered']
casual = we_h['casual']
we_h = Genetic_P(we_h.drop(['registered','casual','dteday'],axis=1),'cnt')
we_h['dteday'] = dates
we_h['registered'] = registered
we_h['casual'] = casual

    |    Population Average   |             Best Individual              |
---- ------------------------- ------------------------------------------ ----------
 Gen   Length          Fitness   Length          Fitness      OOB Fitness  Time Left
   0     8.25 0.13477760559378754       14 0.7195797357054046 0.6807708274465034     28.43s
   1     6.57 0.42967872656510614        8 0.7715210751875244 0.7894675672383449     44.37s
   2     5.37 0.5928136632820061        7 0.8079941814053895 0.790079301577266     46.05s
   3     9.27 0.589066812634625       13 0.8136750060751488 0.8121454467948089     46.60s
   4     9.75 0.6204198049579746       16 0.8394852069165412 0.8512785550730868     45.81s
   5    11.59 0.6313879871102271       28 0.8633665317487316 0.8586679034984442     44.45s
   6     17.7 0.6640623975647103       58 0.8772489481130269 0.887152521690373     43.04s
   7    25.39 0.7250003982369799       52 0.8843299206959049 0.8709346435306187     41.51s
   8    31.72 0.745154035745

# Final Datasets

In [12]:
wd_h.head()

,windspeed,yr,temp,RushHour-Low,hum,cnt,relative_windspeed,relative_hum,atemp,workingday,...,8,9,10,11,12,13,14,dteday,registered,casual
47,0.3582,0,0.22,1,0.44,5,1.319830,-2.120149,0.1970,1,...,0.7316,0.4632,0.4632,0.5864,0.4632,0.4632,0.672552,2011-01-03,5,0
48,0.4179,0,0.20,1,0.44,2,1.788473,-2.004706,0.1667,1,...,0.7600,0.5200,0.5200,0.6400,0.5200,0.5200,0.712000,2011-01-03,2,0
49,0.3881,0,0.16,1,0.47,1,1.467279,-1.715317,0.1364,1,...,0.8144,0.6288,0.6288,0.7376,0.6288,0.6288,0.784704,2011-01-03,1,0
50,0.2836,0,0.16,1,0.47,3,0.565362,-1.649606,0.1364,1,...,0.8144,0.6288,0.6288,0.7376,0.6288,0.6288,0.784704,2011-01-03,3,0
51,0.3881,0,0.14,0,0.50,30,1.420623,-1.406539,0.1061,1,...,-0.1596,-0.3192,-0.3192,-0.2184,-0.3192,-0.3192,-0.181944,2011-01-03,30,0


In [13]:
we_h.head()

,windspeed,yr,temp,RushHour-Low,hum,cnt,relative_windspeed,relative_hum,atemp,workingday,...,8,9,10,11,12,13,14,dteday,registered,casual
6,0.0000,0,0.22,1,0.80,2,-0.408248,0.792594,0.2727,0,...,0.980302,2.980302,2.980302,2.980302,0.980302,1.980302,1.760302,2011-01-01,0,2
7,0.0000,0,0.20,1,0.86,3,-0.377964,2.828427,0.2576,0,...,0.940302,2.940302,2.940302,2.940302,0.940302,1.940302,1.740302,2011-01-01,2,1
8,0.0000,0,0.24,0,0.75,8,-0.353553,-1.037749,0.2879,0,...,1.480000,3.480000,3.480000,3.480000,1.480000,2.480000,2.240000,2011-01-01,7,1
9,0.0000,0,0.32,0,0.76,14,-0.333333,-0.664784,0.3485,0,...,1.640000,3.640000,3.640000,3.640000,1.640000,2.640000,2.320000,2011-01-01,6,8
10,0.2537,0,0.38,0,0.76,36,8.637677,-0.619389,0.3939,0,...,1.920302,4.080605,4.080605,4.080605,2.380000,3.540302,2.540302,2011-01-01,24,12


## Save Both datasets

In [18]:
wd_h.to_csv("weekdays_data_prepared.csv", index=False)
we_h.to_csv("weekends_data_prepared.csv", index=False)